In [12]:
# generate a random number between 0 and 2,147,483,647
Get-Random
(Get-Random) % 10 + 1 # generates a number between 1 and 10
Get-Random -Minimum 1 -Maximum 10 # also generates a number between 1 and 10

1879696599
5
1


In [17]:
# choose a random array value
"rock", "paper", "scissors" | Get-Random

paper


In [20]:
# simple function
function Write-Hello {
    Write-Host "Hello"
}
Write-Hello
Write-Hello

Hello
Hello


In [22]:
# a useful example
function Start-PSAdmin {
    Start-Process PowerShell -Verb RunAs
}
Start-PSAdmin

In [26]:
function Write-Hello {
    # child scope
    # this variable will use a variable defined in the 'parent' scope
    Write-Host $greeting

    $greeting = "Hello"
    Write-Host "child: $greeting"
}
# parent scope
$greeting = "Bonjourno"
Write-Host $greeting
Write-Hello # first time, value is changed within the child scope
Write-Hello # what happens now??

Bonjourno
Bonjourno
child: Hello
Bonjourno
child: Hello


In [29]:
function Write-Hello {
    # child scope
    # this variable will use a variable defined in the 'parent' scope
    Write-Host "1st call: ${script:greeting}"

    $script:greeting = "Hello"
    Write-Host "child: ${script:greeting}"
}
# parent scope
$greeting = "Bonjourno"
Write-Host $greeting
Write-Hello # first time, value is changed within the child scope
Write-Hello # what happens now??

Bonjourno
1st call: Bonjourno
child: Hello
1st call: Hello
child: Hello


In [ ]:
# parameters (simple approach)
function Write-Hello ($Greeting, $Name) { # parameter uses "Pascal Casing" instead of "Camel Casing"
    Write-Host "$Greeting, $Name"
}
Write-Hello -Greeting "Hello" -Name "Jack"
Write-Hello "Bonjour" "Jill"
Write-Hello -Name "John" -Greeting "G'day"

Hello, Jack
Bonjour, Jill
G'day, John


In [42]:
# parameters (powershell approach)
function Add {
    param (
        [Parameter(Mandatory=$true)]
        [int]$a,
        [Parameter(Mandatory=$true)]
        [ValidateRange(0,100)]
        [int]$b
    )
    $a + $b
}
Add 1 2
Add 2 200

3
Add: 
Line |
  13 |  Add 2 200
     |        ~~~
     | Cannot validate argument on parameter 'b'. The 200 argument is greater than the maximum allowed range of 100. Supply an argument that is less than or equal to 100 and then try the command again.


Error: Command failed: SubmitCode: # parameters (powershell approach) ...

In [1]:
# positional parameters
function Add {
    $sum = 0
    foreach ($arg in $args) {
        $sum += $arg
    }
    $sum
}
Add 1 2 3 4 5 6 7 8 9

45


In [9]:
# switch parameters
function Greeting {
    param (
        [switch]$hello
    )

    if ($hello) {
        # $hello is True
        Write-Host "Hello"
    } else {
        Write-Host "Goodbye"
    }
}
Greeting -hello
Greeting
Greeting -hello:$false

Hello
Goodbye
Goodbye


In [10]:
# passing parameters by value
function PassByValue {
    param (
        $integer,
        $decimal,
        $boolean,
        $string
    )

    $integer++
    $decimal *= [Math]::PI
    $boolean = -not $boolean
    $string = "a different string"

}

$i = 10
$f = 1.0
$b = $false
$s = "a string"

Write-Host "before: i = $i, f = $f, b = $b, s = $s"
PassByValue $i $f $b $s
Write-Host "after: i = $i, f = $f, b = $b, s = $s"

before: i = 10, f = 1, b = False, s = a string
after: i = 10, f = 1, b = False, s = a string


In [26]:
# pass by reference
function PassByReference {
    param (
        $array,
        $hashTable,
        [ref]$integer,
        $s
    )
    $array[0] = 234 # modify the original array
    $array += 999   # create a new array local to this function
    $array[0] = 456 # modifies the new array, not the old
    $hashTable['apple'] = 'green' # modifies the original hash table
    $hashTable.orange = 'orange'  # still modifies the original one
    $integer.Value++
    $s = "a new string!"

    $s, $array
}

$a = 1, 2, 3, 4, 5
$h = @{ apple = 'red'; banana = 'yellow' }
$i = 0
$s = "hello!"

Write-Host "before: s = $s, i = $i, a = [$a], h = $($h | ConvertTo-Json -Compress)"
$s, $a = PassByReference $a $h ([ref]$i) $s
Write-Host "after: s = $s, i = $i, a = [$a], h = $($h | ConvertTo-Json -Compress)"


before: s = hello!, i = 0, a = [1 2 3 4 5], h = {"apple":"red","banana":"yellow"}
after: s = a new string!, i = 1, a = [456 2 3 4 5 999], h = {"orange":"orange","banana":"yellow","apple":"green"}


In [35]:
function Get-Math {
    param (
        [int]$a,
        [int]$b
    )
    $a + $b
    $a - $b
    $a * $b
    $a / $b
}
$s, $d, $p, $div = Get-Math -a 2 -b 1
Write-Host "sum = $s, diff = $d, prod = $p, div = $div"
$a = Get-Math -a 2 -b 1
$a

sum = 3, diff = 1, prod = 2, div = 2
3
1
2
2


In [5]:
function Write-Green ($text, $percent) { "{0}: `e[32m{1:P0}`e[30m" -f $text, $percent }
function Write-Yellow ($text, $percent) { "{0}: `e[33m{1:P0}`e[30m" -f $text, $percent }
function Write-Red ($text, $percent) { "{0}: `e[31m{1:P0}`e[30m" -f $text, $percent }

function Calculate-FinalGrade {
    param (
        [Parameter(Mandatory=$true)]
        [ValidateCount(5,5)]
        [double[]]$Assignments,
        [Parameter(Mandatory=$true)]
        [ValidateCount(3,3)]
        [double[]]$Quizzes,
        [Parameter(Mandatory=$true)]
        [ValidateCount(2,2)]
        [double[]]$Exams
    )
    # calculate grade
    $assignmentGrade = ($Assignments | Measure-Object -Average).Average * .45
    $quizGrade = ($Quizzes | Measure-Object -Average).Average * .25
    $examGrade = $Exams[0] * 0.15 + $Exams[1] * 0.15

    $assignmentGrade + $quizGrade + $examGrade
}

# assignments
# original: (6 @ 7.5% = 45%)
# revised: (5 @ 9% = 45%)
$assignments = (0/35), (23/29), (25/30), (21/28), (29/38)

# quizzes
# original: (4 @ 6.3% = 25%)
# revised: (3 @ 8.3% = 25%)
$quizzes = (21/32), (17/27), (20/30)

# exams
# unchanged: (2 @ 15% = 30%)
$exams = (33/51), (33/51)

# final grade (pre-optional)
$final = Calculate-FinalGrade -Assignments $assignments -Quizzes $quizzes -Exams $exams
Write-Yellow "Final grade (pre-optional)" $final

# optional
# replace the lowest assignment grade
# if lower than the optional grade
$optional = (32/32)

# replace lowest grade in assignments, if optional is higher
$assignments = $assignments | Sort-Object
$minGrade = $assignments[0]
if ($optional -gt $minGrade) {
    $assignments[0] = $optional
}

# recalculate final grade (post-optional)
$final = Calculate-FinalGrade -Assignments $assignments -Quizzes $quizzes -Exams $exams

# write summary
if ($optional -gt $minGrade) {
    Write-Red "Lowest assignment" $minGrade
    Write-Green "*Optional assignment" $optional
    Write-Host "*Optional assignment grade will be used."
    Write-Green "Final grade (post-optional)" $final
} else {
    Write-Green "*Lowest assignment" $minGrade
    Write-Red "Optional assignment" $optional
    Write-Host "*Optional assignment grade will not be used."
    Write-Green "Final grade (no change)" $final
}


Final grade (pre-optional): 64%
Lowest assignment: 0%
*Optional assignment: 100%
*Optional assignment grade will be used.
Final grade (post-optional): 73%
